下载模块:
pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken

下载实验数据文本：注意官网的链接可能出现  Could not resolve host: raw.githubusercontent.com
解决： raw.githubusercontent.com换成raw.gitmirror

curl "https://raw.gitmirror.com/Azure-Samples/Azure-OpenAI-Docs-Samples/main/Samples/Tutorials/Embeddings/data/bill_sum_data.csv" --output bill_sum_data.csv

In [33]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
import tiktoken

In [34]:
filePath = "./static" #文件夹路径
df=pd.read_csv(os.path.join(filePath,'bill_sum_data.csv')) # This assumes that you have placed the bill_sum_data.csv in the same directory you are running Jupyter Notebooks
# df

In [35]:
df_bills = df[['text', 'summary', 'title']] #节选'text', 'summary', 'title'
#输出 df_bills
# df_bills 

In [36]:
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s
#对text字段数据进行格式处理
df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))

In [37]:
#token encode处理,删除长度大于8192的参数，因为模型处理长度有限
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]

# df_bills.head(2) 输出前面2行
# len(df_bills)  只有20条符合要求

In [38]:
#分词
sample_encode = tokenizer.encode(df_bills.text[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
# decode
# df_bills.text[0]

In [39]:
#azure open ai的key和endpoint参数
API_KEY = "fced3f5d1b5a4b139d33537d44da6fbc"
RESOURCE_ENDPOINT = "https://open-ai-test-standard-01.openai.azure.com/" 

client = AzureOpenAI(
  api_key = API_KEY,  
  api_version = "2023-05-15",
  azure_endpoint = RESOURCE_ENDPOINT
)

#主要方法，调用azure embedding 模型
def generate_embeddings(text, model="azure_embedding_ada_model"): # model = "deployment_name"
    text = normalize_text(text)
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [40]:
#用lambda分别将每个test内容矢量化,存在ada_v2字段
df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'azure_embedding_ada_model')) # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
# df_bills

In [ ]:
#矢量搜索查询方法

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="azure_embedding_ada_model" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_bills, "Can I get information on cable company tax revenue?", top_n=4)

In [ ]:
res["summary"][9]